In [1]:
#basic libraries 

import pandas as pd 
import numpy as np


import seaborn as sns
sns.set_palette("colorblind")
from matplotlib import pyplot as plt
%matplotlib inline
fig = plt.figure(figsize=(20,5))



import os
import datetime
import warnings
warnings.filterwarnings("ignore")



print(os.getcwd())


/Users/janechoi/Desktop/CONTEST/DATACAMP


<Figure size 1440x360 with 0 Axes>

In [109]:
# # train = pd.read_csv('sentiment_dtm_train.csv',encoding='CP949')

# def load_dataset(traindir, testdir):
#     train = pd.read_csv(traindir,encoding='CP949')
#     test = pd.read_csv(testdir,encoding='CP949')
    
#     #ID not needed
# #     train.drop('Id', axis =1 , inplace=True )
# #     test.drop('Id', axis= 1, inplace=True)
    
#     return train, test 


# train, test =load_dataset('sentiment_dtm_train.csv', 'sentiment_dtm_test.csv')
# test.head()

# # train.sum(axis = 1)

# # train.columns #9452 rows × 302 columns (id, label)

# train.shape
# train
# train['label'].value_counts() #5402, 4050
# train

# from sklearn.model_selection import StratifiedKFold
# from sklearn.model_selection import train_test_split

# avoid_cols = ['label', 'Id']

# cols = [col for col in train.columns if col not in avoid_cols]
# len(cols) #300
# skf = list(StratifiedKFold(n_splits=5,random_state=1008).split(train, train['label'])) 

# X_train, X_val, y_train, y_val = train_test_split(train[cols], 

#                                                     train['label'] , 

#                                                     test_size=0.3, 

#                                                     stratify=train['label'], 

#                                                     random_state=1004)




In [79]:
#FINAL LOADING CODE 


def load_dataset(traindir, testdir):
    train = pd.read_csv(traindir,encoding='CP949')
    test = pd.read_csv(testdir,encoding='CP949')
    
    #ID not needed
#     train.drop('Id', axis =1 , inplace=True)
#     test.drop('Id', axis= 1, inplace=True)

    import copy
    train['w_cnt'] = train[train.columns.difference(['Id'])].sum(axis = 1)
    train['w_perc'] = train['w_cnt'].copy()/300
    test['w_cnt'] = test[test.columns.difference(['Id'])].sum(axis = 1)
    test['w_perc'] = test['w_cnt'].copy()/300
    
    return train, test 

train, test =load_dataset('sentiment_dtm_train.csv', 'sentiment_dtm_test.csv')

train.shape, test.shape #((9452, 302), (9000, 301))


from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

avoid_cols = ['label', 'Id']

cols = [col for col in train.columns if col not in avoid_cols]
len(cols) #300

X_train, X_val, y_train, y_val = train_test_split(train[cols], 

                                                    train['label'] , 

                                                    test_size=0.3, 

                                                    stratify=train['label'], 

                                                    random_state=1004)




In [65]:
#simple train model 


from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix 
import lightgbm as lgb 
import catboost 
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score


models = [
    KNeighborsClassifier(3),
    SVC(probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
	AdaBoostClassifier(),
    GradientBoostingClassifier(),
    GaussianNB(),
    LogisticRegression()]
    

names = ["KNN", "SVC", "DT", "RF", "ADA", "GB", "Gaussian", "LR",]


# define cross validation strategy

def accu_cv(model,X,y):
    acc = np.sqrt(cross_val_score(model, X, y, cv=skf, scoring = 'accuracy'))
    return acc

for name, model in zip(names, models):
    score = accu_cv(model, train[cols], train['label'] )
    print("{}: {:.6f}, {:.4f}".format(name,score.mean(),score.std()))
    

# KNN: 0.861597, 0.0056

# SVC: 0.962360, 0.0039

# DT: 0.895958, 0.0074

# RF: 0.925768, 0.0045

# ADA: 0.895029, 0.0049
# GB: 0.898286, 0.0014
# Gaussian: 0.879298, 0.0020

# LR: 0.983678, 0.0021
  
    
###SVC , RF , LR 

KNN: 0.861597, 0.0056
SVC: 0.962360, 0.0039
DT: 0.895958, 0.0074
RF: 0.925768, 0.0045
ADA: 0.895029, 0.0049
GB: 0.898286, 0.0014
Gaussian: 0.879298, 0.0020
LR: 0.983678, 0.0021


In [81]:
#xgboost -bayesian 


import xgboost as xgb
from bayes_opt import BayesianOptimization


#BAYESIAN OPTIMIZATION 


def xgb_eval(max_depth,learning_rate, n_estimators, reg_lambda, reg_alpha ):
    
    
    skf = list(StratifiedKFold(n_splits=5,random_state=1004).split(X_train, y_train) )
    
    train = X_train[cols]
    val = X_val[cols]
    
    #select params to use 
    params = {'booster' : 'gbtree',
            'objective' :'binary:logistic',
             'eval_metric' : "error", # #(wrong cases)/#(all cases). 
            'random_state' : 314,  
            'verbosity' :2, # 0 (silent), 1 (warning), 2 (info), 3 (debug)
             'silent' : False,
             'nthread':3 }
    
    #optimize parametes :: 
    params["max_depth"] = int(round(max_depth))
    params['learning_rate'] = learning_rate
    params['n_estimators'] = int(round(n_estimators))
    params['reg_lambda'] = max(reg_lambda, 0)
    params['reg_alpha'] = max(reg_alpha, 0)
 

    #use cv to get result 
    
    
    res =  xgb.XGBClassifier(**params)
    best=[]
    
    
    #use skf for 5 fold seperatly from train data (not validation data )
    for fold_, (trn_, val_) in enumerate(skf):
        print("Fold:",fold_)
    
        train_x, train_y = train.iloc[trn_], y_train.iloc[trn_]
        other_x, other_y = train.iloc[val_], y_train.iloc[val_]
        res.fit(train_x, train_y ,eval_set = [(val, y_val)],verbose=False)
    
        pred= res.predict(other_x)
    
        error =accuracy_score(other_y, pred) 
        best.append(error)
        
#     print(best)
    mean = np.mean(np.array(best))
    
#     print(mean)
    return  -(mean) #maximize :: minimize minus mae 


xgbBO = BayesianOptimization(xgb_eval, {'max_depth':(5,20),
                                        'n_estimators':(500,1000), 
                                        'learning_rate':(0.04,0.1), #default 0.3 
                                        'reg_lambda':(0.5,0.7), 
                                        'reg_alpha' :(0.5,0.8)},
                                        random_state= 0 )

#randomly use 10 initial points, and use bayesian for 100 more rounds 
xgbBO.maximize(init_points=2, n_iter=5)  
xgbBO.max 



#prediction for xgboost 

from sklearn.metrics import mean_absolute_error
from xgboost import XGBClassifier
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score,StratifiedKFold

# |   iter    |  target   | learni... | max_depth | n_esti... | reg_alpha | reg_la... |
#  -0.8602   |  0.08804  |  5.22     |  500.1    |  0.557    |  0.5321   |
avoid_cols = ['label', 'Id']

cols = [col for col in train.columns if col not in avoid_cols]
len(cols) #300



def xgb_cv(c=None,v=None):
    
    
    skf = list(StratifiedKFold(n_splits=5,random_state=1004).split(X_train, y_train) )
    
    train = X_train[cols]
    val = X_val[cols]
    
    params={}
    #select params to use 
    basicparams = {'booster' : 'gbtree',
            'objective' :'binary:logistic',
             'eval_metric' : "error", # #(wrong cases)/#(all cases). 
            'random_state' : 314,  
            'verbosity' :2, # 0 (silent), 1 (warning), 2 (info), 3 (debug)
             'silent' : False,
             'nthread':3 }
    
    #optimize parametes :: 
    params["max_depth"] = int(round(5.22 ))
    params['learning_rate'] =  0.08804  
    params['n_estimators'] = int(round( 500.1))
    params['reg_lambda'] = max( 0.5321 , 0)
    params['reg_alpha'] = max( 0.557 , 0)
 
    params.update(basicparams)
    
    #fit on xgboost regressor 
    train_x =X_train[cols]
    val = X_val[cols]
    
    res = xgb.XGBClassifier(**params)
    res.fit(train_x, y_train ,eval_set = [(val, y_val)],verbose=False)
    
    #predictions 
    predictions = res.predict(X_val[cols])
    error= accuracy_score(y_val, predictions)
    
    print('AC for validation set is...' + str(error))

    #AC for validation set is...0.8515514809590973
    return  res
    

res= xgb_cv()





|   iter    |  target   | learni... | max_depth | n_esti... | reg_alpha | reg_la... |
-------------------------------------------------------------------------------------
Fold: 0
Fold: 1
Fold: 2
Fold: 3
Fold: 4
|  1        | -0.8868   |  0.07293  |  15.73    |  801.4    |  0.6635   |  0.5847   |
Fold: 0
Fold: 1
Fold: 2
Fold: 3
Fold: 4
|  2        | -0.8878   |  0.07875  |  11.56    |  945.9    |  0.7891   |  0.5767   |
Fold: 0
Fold: 1
Fold: 2
Fold: 3
Fold: 4
|  3        | -0.8602   |  0.08804  |  5.22     |  500.1    |  0.557    |  0.5321   |
Fold: 0
Fold: 1


KeyboardInterrupt: 

In [ ]:
#CATBOOST 
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from scipy.stats import randint
from random import uniform


## split by stratifiedfold
xy_train = pd.concat([X_train, y_train], axis=1)
skf = list(StratifiedKFold(n_splits=5,random_state=1008).split(xy_train, xy_train['label'])) 

from sklearn.metrics import accuracy_score

def catboost_pipline(X_train_data, X_test_data, y_train_data, y_test_data, 
                     model, param_dist, cv=5, scoring_fit = 'accuracy'):

    n_iter_search = 10
    rs = RandomizedSearchCV(estimator = model, 
                            param_distributions = param_dist,
                            n_iter = n_iter_search,
                            cv = skf,
                            n_jobs = 1,
                            scoring = scoring_fit,
                            random_state = 314,
                            verbose = 10,
                            return_train_score = True
                           )

    fitted_model = rs.fit(X_train_data, y_train_data,
                          verbose = False,
                          eval_set = (X_test_data, y_test_data),
                          early_stopping_rounds = 100,
#                           metric = 'Accuracy'
                         )
    
    # early stopping rounds, eval_set은 한쌍으로
    # early stopping rounds의 기준은 eval set임
    print("BEST PARAMS : ", rs.best_params_)
    print("BEST SCORE : ", rs.best_score_)

    pred = fitted_model.predict(X_test_data)
    
    return fitted_model, pred, rs


avoid_cols = ['Id','label']
cols = [col for col in X_train.columns if col not in avoid_cols] #store_id, 3,6,12month, month 

# cat = [X_train.columns.difference(['w_cnt', 'w_perc'])]
cat = X_train.columns.difference(['w_cnt', 'w_perc']).to_numpy()

param_dist = {'learning_rate' : np.linspace(0.05, 0.1), #uniform(0.05, 0.1)
              'depth' : randint(5, 100),
              'iterations' : randint(1000, 3000),
              'l2_leaf_reg': np.linspace(0.2, 0.9),
              'bagging_temperature' : randint(0, 1000),
#               'border_count' : randint(5, 254)
             }

model = CatBoostClassifier(cat_features = cat,
                          random_seed = 314,
                          verbose = True,
                          use_best_model = True,
                          eval_metric = 'Accuracy',
                          od_type = 'Iter',
                          od_wait = 100,
#                           bootstrap_type = 'Bayesian',
#                           boosting_type = 'Ordered'
                         )
                          

m, pred, rs = catboost_pipline(X_train[cols], X_val[cols], 
                               y_train, y_val, 
                               model, param_dist)

In [103]:
#SVC / LR /RF --using validation set 

from sklearn.metrics import accuracy_score

svc = SVC(probability=True)
svc.fit(X_train,y_train)
svc_result =svc.predict(X_val)

print(accuracy_score(y_val, svc_result))
#0.9167842031029619



lr = LogisticRegression()
lr.fit(X_train,y_train)
lr_result =lr.predict(X_val)

print(accuracy_score(y_val, lr_result) )
#0.9594499294781382


rf = RandomForestClassifier()
rf.fit(X_train,y_train)

rf_result =rf.predict(X_val)

print(accuracy_score(y_val, rf_result) )

0.9167842031029619
0.9594499294781382
0.8430888575458392


In [104]:
#making submission file 

avoid_cols2 = [ 'Id']

cols2 = [col for col in test.columns if col not in avoid_cols2]

def submission(model, test):

    #predictions 
    y_pred = model.predict(test[cols2])
    submission_df = pd.DataFrame()
    submission_df['Id'] = test['Id'] #ID
    submission_df['label'] = y_pred
    
    return submission_df 

svc_test = submission(svc,test)
lr_test = submission(lr,test)
rf_test = submission(rf,test)
xgb_test = submission(xgb,test)
xgb_test.to_csv('rf_sub.csv', index=0)


svc_test.to_csv('svc_sub.csv', index=0)
lr_test.to_csv('lr_sub.csv', index=0)
rf_test.to_csv('rf_sub.csv', index=0)